# Charts for mojort paper

## Boiler plates

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from benchkit.charts.printedcharts import export_figure
import matplotlib.pyplot as plt
import benchkit
from benchkit.charts.dataframes import get_dataframe
import os
from pathlib import Path
from benchkit.utils.dir import gitmainrootdir
import re

In [ ]:
repo_dir = gitmainrootdir()
paper_dir = Path("/tmp")

fig_dir = paper_dir / "figures/generated"

os.makedirs(str(fig_dir), exist_ok=True)

In [ ]:
def make_parent(
    path: Path,
) -> None:
    parent_dir = Path(path).parent.resolve()
    parent_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
def export_fig(
    plot,
    path: Path,
) -> None:
    make_parent(path)

    export_figure(
        plot=plot,
        path=path,
        creation_year=2024,
        creation_month=12,
        creation_day=10,
    )


In [ ]:
CONTEXT = "talk"
STYLE = "whitegrid"
PALETTE = "colorblind"
# ESTIMATOR = np.median
ESTIMATOR_STR = "median"
# CONFINT = 'sd'
FIG_USE_LATEX_FONTS = True


In [ ]:
def set_config(width=8, height=6, font_scale=1.15):
    sns.set_theme(
        context=CONTEXT,
        style=STYLE,
        palette=PALETTE,
        font_scale=font_scale,
        rc={
            "figure.figsize": (width, height),
            "pdf.fonttype": 42,
            "pdf.use14corefonts": True,
            "text.usetex": FIG_USE_LATEX_FONTS,
        },
    )


In [ ]:
set_config()


## Microbenches

In [ ]:
path_results = Path("results/microbenches/")
csv_paths = [path_results / f for f in os.listdir(path_results) if f.endswith(".csv")]
dataframes = [get_dataframe(p) for p in sorted(csv_paths)]
# dataframes = dataframes[::-1] # reverse data frame order
dataframe = pd.concat(dataframes)
df = dataframe


In [ ]:
df["src_filename"] = (
    df["src_filename"]
    .str.replace("knmp", "String search (KMP)", regex=True)
    .str.replace("mandelbrot", "Mandelbrot", regex=True)
    .str.replace("matmul", "Matrix multiplication", regex=True)
)

In [ ]:
df["runtime"] /= 1000.0 # µs to ms


In [ ]:
# Ensure required columns
required = {"language","src_filename","size","runtime"}
missing = required - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# Parse language into family + opt level
def split_lang(s: str):
    m = re.match(r"^\s*([A-Za-z\+\-]+(?:\s*[A-Za-z\-]+)?)\s*(?:(-O[0-3]|-Ofast))?\s*$", str(s))
    if not m:
        return s.strip(), "unspecified"
    fam = m.group(1).strip()
    opt = (m.group(2) or "unspecified").strip()
    return fam, opt

fam_opt = df["language"].apply(split_lang)
df["lang_family"] = fam_opt.str[0]
df["opt_level"]   = fam_opt.str[1]

In [ ]:
df["lang_family"] = (
    df["lang_family"]
    .str.replace("c-clang", "C (Clang)", regex=True)
    .str.replace("c-gcc", "C (GCC)", regex=True)
    .str.replace("cpp-clang", "C++ (Clang)", regex=True)
    .str.replace("cpp-gcc", "C++ (GCC)", regex=True)
    .str.replace("mojo", "Mojo", regex=True)
    .str.replace("rust", "Rust", regex=True)
)

set(df["lang_family"])

In [ ]:
# Orders for plotting
opt_order  = ["-O1", "-O2", "-O3"]
size_order = sorted(df["size"].dropna().unique())
lang_order = sorted(df["lang_family"].dropna().unique(),
                    key=lambda s: ("mojo" not in s.lower(), s.lower()))

# Cast before slicing
df["opt_level"]   = pd.Categorical(df["opt_level"],   categories=opt_order, ordered=True)
df["size"]        = pd.Categorical(df["size"],        categories=size_order, ordered=True)
df["lang_family"] = pd.Categorical(df["lang_family"], categories=lang_order, ordered=False)

# Now filter
df = df[~df["opt_level"].isin(["unspecified", "-Ofast"])]

In [ ]:
plot_df = (
    df.loc[df["opt_level"].isin(opt_order)]
      .dropna(subset=["src_filename", "size", "lang_family", "runtime", "opt_level"])
      .copy()
)

# Reset index to avoid duplicate-label issues inside seaborn
plot_df.reset_index(drop=True, inplace=True)

# Flatten (src_filename, size) into one panel column
plot_df["panel"] = (
    plot_df["src_filename"].astype(str)
    + " — size "
    + plot_df["size"].astype(str)
)

plot_df["size_num"] = pd.to_numeric(plot_df["size"].astype(str), errors="coerce")
panel_order = (plot_df[["panel", "src_filename", "size_num"]]
               .drop_duplicates()
               .sort_values(["src_filename", "size_num"])["panel"].tolist())

local_lang_order = [x for x in lang_order if x in plot_df["lang_family"].unique()]


In [ ]:
g = sns.catplot(
    data=plot_df,
    x="opt_level", y="runtime",
    hue="lang_family",
    col="panel", col_order=panel_order, col_wrap=2,
    order=opt_order, hue_order=local_lang_order,
    kind="bar",
    errorbar=("pi", 95),
    capsize=0.15,
    sharey=False,
    height=3.8, aspect=2.0,
)

g.set_axis_labels("Optimization level", "Runtime (ms)")
g.set_titles("{col_name}")
#g.add_legend(title="Language")
# Legend: remove title, put on top in one row
sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(.45, -0.04), ncol=6, title=None, frameon=False,
)

#plt.tight_layout()
plt.show()


In [ ]:
export_fig(g, fig_dir / "microbenches.pdf")
export_fig(g, Path("/tmp/microbenches.png"))

## cyclictest

In [ ]:
def remap_results_path(src: Path, new_root: Path) -> Path:
    """
    Remap a full path under an unknown prefix that contains 'results/benchmark_*'
    to a new root, preserving the tail after 'results/'.

    Example:
        /home/.../results/benchmark_xxx/... -> ./results/cyclictest/benchmark_xxx/...
    """
    parts = src.parts
    try:
        # find the index of 'results'
        idx = parts.index('results')
    except ValueError:
        raise ValueError(f"'results' directory not found in path: {src}")

    # build relative path *after* 'results/'
    rel = Path(*parts[idx + 1:])

    # reattach under new root
    return new_root / rel

In [ ]:
def gen_cyclictest_chart(path_results: Path, chart_dst_filename: Path) -> None:
    csv_paths = [path_results / f for f in os.listdir(path_results) if f.endswith(".csv")]
    dataframes = [get_dataframe(p) for p in sorted(csv_paths)]
    
    dataframe = pd.concat(dataframes)

    dataframe["language"] = (
        dataframe["language"]
        .str.replace("mojo", "Mojo", regex=True)
        .str.replace("c", "C", regex=True)
    )
    
    frames = []
    path_col = "datapath"
    value_col = "latency"
    for _, row in dataframe.iterrows():
        path = Path(row[path_col])
        path = remap_results_path(src=path, new_root=path_results)
        with path.open("r") as f:
            for line in f:
                if not line.strip():
                    continue
                new_row = row.to_dict()
                new_row[value_col] = float(line.strip())
                frames.append(new_row)
    
    df = pd.DataFrame(frames)
    
    lkv = list(set(df["kernel_version"]))
    assert 1 == len(lkv)
    kernel_version = lkv[0]
    kernel_version
    
    g = sns.catplot(
        data=df,
        kind="violin",
        y="latency",
        x="threads",
        hue="language",
        split=True,
        inner="quart",
        height=7,
        aspect=1.5,
        width=0.9,
        #legend=False,
    )
    
    # Axis labels
    g.set_axis_labels("Number of cyclictest threads", "Cyclictest latency (ns)")
    
    # Main title (instead of set_titles)
    g.fig.suptitle(
        f"Comparing latencies of C and Mojo for cyclictest - {kernel_version} kernel",
        y=1.02  # push title a bit up
    )
    
    # Legend: remove title, put on top in one row
    sns.move_legend(
        g,
        "lower center",
        bbox_to_anchor=(.25, 0.88),
        ncol=2,
        title=None,
        frameon=False,
    )
    
    plt.show()

    export_fig(g, chart_dst_filename)

    return dataframe

In [ ]:
df = gen_cyclictest_chart(
    path_results=Path("results/cyclictest.nrt/"),
    chart_dst_filename=fig_dir / "cyclic-nrt.pdf",
)

In [ ]:
gen_cyclictest_chart(
    path_results=Path("results/cyclictest.rt/"),
    chart_dst_filename=fig_dir / "cyclic-rt.pdf",
)

# Binary file size

In [ ]:
path_results = Path("results/size")
csv_paths = [path_results / f for f in os.listdir(path_results) if f.endswith(".csv")]
dataframes = [get_dataframe(p) for p in sorted(csv_paths)]
dataframe = pd.concat(dataframes)
df = dataframe

df["src_filename"] = (
    df["src_filename"]
    .str.replace("knmp", "String search (KMP)", regex=True)
    .str.replace("mandelbrot", "Mandelbrot", regex=True)
    .str.replace("matmul", "Matrix multiplication", regex=True)
)

In [ ]:
# Ensure required columns are present
required = {"language","src_filename","filesize"}
missing = required - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# Parse language into family + opt level, without losing raw rows
def split_lang(s: str):
    m = re.match(r"^\s*([A-Za-z\+\-]+(?:\s*[A-Za-z\-]+)?)\s*(?:(-O[0-3]|-Ofast))?\s*$", str(s))
    if not m:
        return s.strip(), "unspecified"
    fam = m.group(1).strip()
    opt = (m.group(2) or "-O0").strip()
    return fam, opt

fam_opt = df["language"].apply(split_lang)
df["lang_family"] = fam_opt.apply(lambda x: x[0])

df["lang_family"] = (
    df["lang_family"]
    .str.replace("c-clang", "C (Clang)", regex=True)
    .str.replace("c-gcc", "C (GCC)", regex=True)
    .str.replace("cpp-clang", "C++ (Clang)", regex=True)
    .str.replace("cpp-gcc", "C++ (GCC)", regex=True)
    .str.replace("mojo", "Mojo", regex=True)
    .str.replace("rust", "Rust", regex=True)
)

set(df["lang_family"])

df["opt_level"]   = fam_opt.apply(lambda x: x[1])

# Orders for nice, consistent axes
opt_order  = ["-O0", "-O3",]
df["lang_family"] = (
    df["lang_family"]
    .str.replace("mojo", "Mojo", regex=True)
    .str.replace("rust", "Rust", regex=True)
)
lang_order = sorted(df["lang_family"].dropna().unique(), key=lambda s: ("mojo" not in s.lower(), s.lower()))

# Cast to categoricals for stable plotting order
df["opt_level"]   = pd.Categorical(df["opt_level"],   categories=opt_order, ordered=True)
df["lang_family"] = pd.Categorical(df["lang_family"], categories=lang_order, ordered=False)

df = df[df["opt_level"] != "unspecified"]
df = df[df["opt_level"] != "-Ofast"]

In [ ]:
g = sns.catplot(
    data=df,
    x="opt_level", y="filesize",
    hue="lang_family",
    col="src_filename",
    order=opt_order, hue_order=lang_order,
    kind="bar",
    errorbar=("pi", 95),  # 95% percentile interval
    capsize=0.15,
    sharey=False,
    height=3.8, aspect=1.2
)
g.set_ylabels('Executable file size (bytes)',fontsize=15)
g.legend.set_title(None)
for ax in g.axes.flat:
        ax.set(yscale="log")

g.set_axis_labels("Optimization level", "Executable size (bytes)")
g.set_titles("{col_name}")

sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(.47, -0.13), ncol=6, title=None, frameon=False,
)

In [ ]:
export_fig(g, fig_dir / "sizes.pdf")

## GPU

In [ ]:
path_results = Path("results/gpu")
csv_paths = [path_results / f for f in os.listdir(path_results) if f.endswith(".csv")]
dataframes = [get_dataframe(p) for p in sorted(csv_paths)]
dataframe = pd.concat(dataframes)
df = dataframe


In [ ]:
df["src_filename"] = (
    df["src_filename"]
    .str.replace("matmul_gpu", "GPU Matrix Multiplication", regex=True)
    .str.replace("matmul", "CPU Matrix Multiplication", regex=True)
)

df["runtime"] /= 1000.0

df


In [ ]:
# Ensure required columns are present
required = {"language","src_filename","runtime"}
missing = required - set(df.columns)
assert not missing, f"Missing columns: {missing}"

# Parse language into family + opt level, without losing raw rows
def split_lang(s: str):
    m = re.match(r"^\s*([A-Za-z\+\-]+(?:\s*[A-Za-z\-]+)?)\s*(?:(-O[0-3]|-Ofast))?\s*$", str(s))
    if not m:
        return s.strip(), "unspecified"
    fam = m.group(1).strip()
    opt = (m.group(2) or "unspecified").strip()
    return fam, opt

fam_opt = df["language"].apply(split_lang)
df["lang_family"] = fam_opt.apply(lambda x: x[0])
df["opt_level"]   = fam_opt.apply(lambda x: x[1])

# Orders for nice, consistent axes
opt_order  = ["-O1", "-O2", "-O3",]
lang_order = sorted(df["lang_family"].dropna().unique(), key=lambda s: ("mojo" not in s.lower(), s.lower()))

# Cast to categoricals for stable plotting order
df["opt_level"]   = pd.Categorical(df["opt_level"],   categories=opt_order, ordered=True)
df["lang_family"] = pd.Categorical(df["lang_family"], categories=lang_order, ordered=False)

df = df[df["opt_level"] != "unspecified"]
df = df[df["opt_level"] != "-Ofast"]


In [ ]:
g = sns.catplot(
    data=df,
    x="size", y="runtime",
    hue="src_filename",
    kind="bar",
    errorbar=("pi", 95),  # 95% percentile interval
    capsize=0.15,
    sharey=False,
    height=4.4, aspect=1.3
)

g.set_axis_labels("Matrix Size", "Runtime (ms)")
sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(0.4, 0.9), ncol=1, title=None, frameon=False,
)


In [ ]:
export_fig(g, fig_dir / "gpu-runtime.pdf")

In [ ]:
g = sns.catplot(
    data=df,
    kind="violin",
    y="runtime",
    col="size",
    hue="src_filename",
    # split=True,
    inner="quart"
)

In [ ]:
# def norm(dataframer):
#     l = []
#     for s in dataframer["size"].unique():
#         dataframe = dataframer[dataframer['size'] == s]
#         for lan in dataframe["src_filename"].unique():
#             mojodf = dataframe[dataframe.src_filename == lan]
#             avg = np.average(mojodf['runtime'])
#             nw = (dataframe[dataframe.src_filename == lan]['runtime'] / avg) - 1
#             dataframe.loc[dataframe["src_filename"] == lan,'normalized'] = nw
#         l.append(dataframe)
#     return pd.concat(l)

# def shiftr(dataframer):
#     l = []
#     for s in dataframer["size"].unique():
#         dataframe = dataframer[dataframer['size'] == s]
#         for lan in dataframe["src_filename"].unique():
#             mojodf = dataframe[dataframe.src_filename == lan]
#             avg = np.average(mojodf['runtime'])
#             nw = (dataframe[dataframe.src_filename == lan]['runtime'] - avg)
#             dataframe.loc[dataframe["src_filename"] == lan,'shifted'] = nw
#         l.append(dataframe)
#     return pd.concat(l)

# nwdf_norm = norm(df)
# nwdf_shift = shiftr(df)

# g = sns.catplot(
#     data=nwdf_norm,
#     kind="violin",
#     y="normalized",
#     col="size",
#     hue="src_filename",
#     split=True,
#     inner="quart"
# )

# g = sns.catplot(
#     data=nwdf_shift,
#     kind="violin",
#     y="shifted",
#     col="size",
#     hue="src_filename",
#     split=True,
#     inner="quart"
# )